In [ ]:
!curl -L "https://public.roboflow.com/ds/kWMeUcWcl6?key=DzvTv5bXEx" > roboflow.zip; unzip roboflow.zip; rm roboflow.zip
#데이터셋 불러오기

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
 extracting: export/images/armas (693)_jpg.rf.539e7f818abfba8b13efc60c3eaa4b09.jpg  
 extracting: export/images/armas (1623)_jpg.rf.54322b09fff9789a2617f07a19f8f531.jpg  
 extracting: export/images/armas (772)_jpg.rf.5335483aeff9c87fd46a3a12e5a75f4d.jpg  
 extracting: export/images/armas (1101)_jpg.rf.53cb10cc7705158adb60227aa482711d.jpg  
 extracting: export/images/armas (623)_jpg.rf.53ee22796f191ca0eaf6124f4e133f5f.jpg  
 extracting: export/images/armas (2278)_jpg.rf.535d90355a01edf613c11d1d6168de3e.jpg  
 extracting: export/images/armas (2254)_jpg.rf.5451063a5496a056897d719fb4ccdc83.jpg  
 extracting: export/images/armas (1306)_jpg.rf.549461e692756d4a158af908e7eb8843.jpg  
 extracting: export/images/armas (2797)_jpg.rf.5518ea2e6baa681a051807d54dae5d9d.jpg  
 extracting: export/images/armas (1947)_jpg.rf.54cd92bf56c54ddc500de583356367a2.jpg  
 extracting: export/images/armas (304)_jpg.rf.54d3beb055395743c1d0df76e3214b6f.jpg  
 extracting: export/im

In [ ]:
%cd /content
!git clone https://github.com/ultralytics/yolov5.git
#YOLO5 설치

/content
Cloning into 'yolov5'...
remote: Enumerating objects: 8754, done.
remote: Counting objects: 100% (468/468), done.
remote: Compressing objects: 100% (279/279), done.
remote: Total 8754 (delta 304), reused 314 (delta 189), pack-reused 8286
Receiving objects: 100% (8754/8754), 9.66 MiB | 20.77 MiB/s, done.
Resolving deltas: 100% (6044/6044), done.


In [ ]:
%cd /content/yolov5/
! pip install -r requirements.txt
#필요한 파일 설치

/content/yolov5
     |████████████████████████████████| 636 kB 7.3 MB/s 
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
%cat /content/dataset/data.yaml
#데이터 정보 읽기

train: ../train/images
val: ../valid/images

nc: 1
names: ['pistol']

In [ ]:
%cd /
from glob import glob

img_list = glob('/content/dataset/export/images/*.jpg')

print(len(img_list))
#사진 갯수파악

/
2971


In [ ]:
from sklearn.model_selection import train_test_split

train_img_list, val_img_list = train_test_split(img_list, test_size=0.2, random_state=2000)

print(len(train_img_list), len(val_img_list))
#TarinSet, TestSet 나누기

2376 595


In [ ]:
with open('content/dataset/train.txt', 'w') as f:
  f.write('\n'.join(train_img_list)+'\n')

with open('content/dataset/val.txt', 'w') as f:
  f.write('\n'.join(val_img_list)+'\n')
  #TarinSet, TestSet을 txt파일로 저장

In [ ]:
import yaml

with open('/content/dataset/data.yaml', 'r') as f:
  data = yaml.load(f)

print(data)

data['train'] = '/content/dataset/train.txt'
data['val'] = '/content/dataset/val.txt'

with open('/content/dataset/data.yaml', 'w') as f:
  yaml.dump(data, f)

print(data)
#yaml파일에 TarinSet, TestSet삽입

{'train': '../train/images', 'val': '../valid/images', 'nc': 1, 'names': ['pistol']}
{'train': '/content/dataset/train.txt', 'val': '/content/dataset/val.txt', 'nc': 1, 'names': ['pistol']}


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  after removing the cwd from sys.path.


In [ ]:
%cd /content/yolov5/

!python train.py --img 416 --batch 16 --epochs 50 --data /content/dataset/data.yaml --cfg ./models/yolov5s.yaml --weights yolov5s.pt --name gun_yolov5s_results
#학습

/content/yolov5
train: weights=yolov5s.pt, cfg=./models/yolov5s.yaml, data=/content/dataset/data.yaml, hyp=data/hyps/hyp.scratch.yaml, epochs=50, batch_size=16, imgsz=416, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, adam=False, sync_bn=False, workers=8, project=runs/train, entity=None, name=gun_yolov5s_results, exist_ok=False, quad=False, linear_lr=False, label_smoothing=0.0, upload_dataset=False, bbox_interval=-1, save_period=-1, artifact_alias=latest, local_rank=-1, freeze=0
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v5.0-351-ge96c74b torch 1.9.0+cu102 CUDA:0 (Tesla T4, 15109.75MB)

hyperparameters: lr0=0.01, lrf=0.2, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl_gamma=0.0, hsv_h=0.015, hsv_s=0.7, 

In [ ]:
from IPython.display import Image
import os

val_img_path = val_img_list[0]

!python detect.py --weights /content/yolov5/runs/train/gun_yolov5s_results/weights/best.pt --img 416 --conf 0.5 --source "{val_img_path}"
#테스트

detect: weights=['/content/yolov5/runs/train/gun_yolov5s_results/weights/best.pt'], source=/content/dataset/export/images/armas (1767)_jpg.rf.dfd97ffc6d6a896d32de4ab7c8b6f983.jpg, imgsz=416, conf_thres=0.5, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False
YOLOv5 🚀 v5.0-351-ge96c74b torch 1.9.0+cu102 CUDA:0 (Tesla T4, 15109.75MB)

Fusing layers... 
Model Summary: 224 layers, 7053910 parameters, 0 gradients, 16.3 GFLOPs
image 1/1 /content/dataset/export/images/armas (1767)_jpg.rf.dfd97ffc6d6a896d32de4ab7c8b6f983.jpg: 416x416 1 pistol, Done. (0.010s)
Results saved to runs/detect/exp
Done. (0.033s)


In [ ]:
import torch
from IPython.display import Image, clear_output  # to display images

clear_output()
print('Setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

Setup complete. Using torch 1.9.0+cu102 _CudaDeviceProperties(name='Tesla T4', major=7, minor=5, total_memory=15109MB, multi_processor_count=40)


In [ ]:
from google.colab import files 
uploaded = files.upload()
#파일올리기

Saving New World Record - Shooting .295 Second at 21 Feet - Cisko Master Gunfighter.mp4 to New World Record - Shooting .295 Second at 21 Feet - Cisko Master Gunfighter.mp4


In [ ]:
!python detect.py --weights /content/yolov5/runs/train/gun_yolov5s_results/weights/best.pt  --source /content/yolov5/pistol.mp4
#영상학습

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
video 1/1 (1025/6022) /content/yolov5/pistol.mp4: 384x640 2 pistols, Done. (0.009s)
video 1/1 (1026/6022) /content/yolov5/pistol.mp4: 384x640 2 pistols, Done. (0.009s)
video 1/1 (1027/6022) /content/yolov5/pistol.mp4: 384x640 2 pistols, Done. (0.009s)
video 1/1 (1028/6022) /content/yolov5/pistol.mp4: 384x640 2 pistols, Done. (0.009s)
video 1/1 (1029/6022) /content/yolov5/pistol.mp4: 384x640 2 pistols, Done. (0.009s)
video 1/1 (1030/6022) /content/yolov5/pistol.mp4: 384x640 2 pistols, Done. (0.009s)
video 1/1 (1031/6022) /content/yolov5/pistol.mp4: 384x640 2 pistols, Done. (0.009s)
video 1/1 (1032/6022) /content/yolov5/pistol.mp4: 384x640 1 pistol, Done. (0.009s)
video 1/1 (1033/6022) /content/yolov5/pistol.mp4: 384x640 1 pistol, Done. (0.009s)
video 1/1 (1034/6022) /content/yolov5/pistol.mp4: 384x640 1 pistol, Done. (0.009s)
video 1/1 (1035/6022) /content/yolov5/pistol.mp4: 384x640 1 pistol, Done. (0.009s)
video 1/1 (1036/6022) /content/yolov5/pistol